# Clustering model

In [21]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!pip install scikit-learn

  Using cached scikit_learn-1.7.2-cp310-cp310-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.15.3-cp310-cp310-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.2-cp310-cp310-macosx_12_0_arm64.whl (8.7 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached scipy-1.15.3-cp310-cp310-macosx_14_0_arm64.whl (22.4 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn]0m 3/4 [scikit-learn]


In [22]:
from pathlib import Path

body_path = "/Users/williamchalons/code/WiwiC/VERA/data/processed/myvideo/df_Body_raw_data.csv"
face_path = "/Users/williamchalons/code/WiwiC/VERA/data/processed/myvideo/df_Face_raw_data.csv"
audio_path = "/Users/williamchalons/code/WiwiC/VERA/data/processed/myvideo/df_Audio_raw_data.csv"

df_body = pd.read_csv(body_path)
df_face = pd.read_csv(face_path)
df_audio = pd.read_csv(audio_path)

In [23]:
df_body.head()
print (df_body.shape)

(1741, 6)


In [24]:
df_face.head()
print (df_face.shape)

(1741, 5)


In [25]:
df_audio.head()
print (df_audio.shape)

(1, 7)


In [26]:
#Dropping first row as there are empty values
df_body = df_body.iloc[1:].reset_index(drop=True)
df_face = df_face.iloc[1:].reset_index(drop=True)

#Dropping columns timestamps and seconds
df_body = df_body.drop(columns=["timestamp", "second"])
df_face = df_face.drop(columns=["timestamp", "second"])


In [27]:
df_body.shape, df_face.shape, df_audio.shape

((1740, 4), (1740, 3), (1, 7))

In [31]:
face_feats = {}

for col in df_face.columns:
    face_feats[f"face_{col}_mean"] = df_face[col].mean()
    face_feats[f"face_{col}_var"]  = df_face[col].var()

In [32]:
face_feats

{'face_head_speed_mean': np.float64(0.030878922789206058),
 'face_head_speed_var': np.float64(0.0005573005966736912),
 'face_gaze_dg_mean': np.float64(0.016534446089880264),
 'face_gaze_dg_var': np.float64(0.00013271276722550212),
 'face_smile_mean': np.float64(0.8222391493167966),
 'face_smile_var': np.float64(0.00298501467927455)}

In [35]:
body_feats = {}

for col in df_body.columns:
    body_feats[f"body_{col}_mean"] = df_body[col].mean()
    body_feats[f"body_{col}_var"]  = df_body[col].var()

In [36]:
body_feats

{'body_gesture_magnitude_mean': np.float64(2.098175306722556),
 'body_gesture_magnitude_var': np.float64(0.37596754527585013),
 'body_gesture_activity_mean': np.float64(0.08828037666589907),
 'body_gesture_activity_var': np.float64(0.0067096928485235515),
 'body_body_sway_mean': np.float64(0.018709107564855516),
 'body_body_sway_var': np.float64(0.00043056235587380243),
 'body_posture_openness_mean': np.float64(179.9999997983233),
 'body_posture_openness_var': np.float64(1.6301772866684967e-13)}

In [37]:
audio_feats = {
    f"audio_{col}": df_audio[col].iloc[0]
    for col in df_audio.columns
}

In [40]:
final_vector = {
    "video_name": 'video_name_to_be_inserted',
    **body_feats,
    **face_feats,
    **audio_feats
}

In [42]:
final_vector

{'video_name': 'video_name_to_be_inserted',
 'body_gesture_magnitude_mean': np.float64(2.098175306722556),
 'body_gesture_magnitude_var': np.float64(0.37596754527585013),
 'body_gesture_activity_mean': np.float64(0.08828037666589907),
 'body_gesture_activity_var': np.float64(0.0067096928485235515),
 'body_body_sway_mean': np.float64(0.018709107564855516),
 'body_body_sway_var': np.float64(0.00043056235587380243),
 'body_posture_openness_mean': np.float64(179.9999997983233),
 'body_posture_openness_var': np.float64(1.6301772866684967e-13),
 'face_head_speed_mean': np.float64(0.030878922789206058),
 'face_head_speed_var': np.float64(0.0005573005966736912),
 'face_gaze_dg_mean': np.float64(0.016534446089880264),
 'face_gaze_dg_var': np.float64(0.00013271276722550212),
 'face_smile_mean': np.float64(0.8222391493167966),
 'face_smile_var': np.float64(0.00298501467927455),
 'audio_wpm': np.float64(169.77225672877847),
 'audio_pause_ratio': np.float64(0.0898296334537945),
 'audio_pitch_mean_h

In [43]:
row_df = pd.DataFrame([final_vector])

In [44]:
import os
master_path = "/Users/williamchalons/code/WiwiC/VERA/data/clustering_dataset/master_vector_data_set.csv"

if os.path.exists(master_path):
    master_df = pd.read_csv(master_path)
else:
    master_df = pd.DataFrame()

EmptyDataError: No columns to parse from file

In [ ]:
video_name = final_vector["video_name"]

if "video_name" in master_df.columns:
    exists = (master_df["video_name"] == video_name).any()
else:
    exists = False

if exists:
    print("Updating existing video vector...")
    master_df = master_df[master_df["video_name"] != video_name]

In [ ]:
master_df = pd.concat([master_df, row_df], ignore_index=True)
master_df.to_csv(master_path, index=False)
print("Saved updated master dataset:", master_path)
